In [3]:
!pip install --q pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 76.1 MB/s eta 0:00:00


In [10]:
import os
import re
import gc
import fitz
import torch
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

torch.cuda.empty_cache()
gc.collect()

device = "cuda" if torch.cuda.is_available else "cpu"

print(f" using {device}")

 using cuda


In [5]:
def text_formater(text: str) -> str:
    return_text = text.replace("\n", " ")
    return_text = return_text.lower()
    return return_text

In [6]:
def open_and_clean_pdf(path1: str, pages_and_text: list, page_number=0):
    pdf = fitz.open(path1)
    for pages_no, pages_text in tqdm(enumerate(pdf)):
        text = pages_text.get_text()
        text = text_formater(text)
        pages_and_text.append(
            {
                "page_number": pages_no - page_number,
                "page_char_count": len(text),
                "number_of_tokens": len(text) / 4,
                "pages_sentence_count": len(text.split(".")),
                "page_text": text,
                "Book_Name": path1.replace(".pdf", " "),
            }
        )

In [7]:
pages_and_text = []
open_and_clean_pdf(
    "/content/Stephen Hawking -A Brief History Of Time.pdf", pages_and_text, 7
)
open_and_clean_pdf("/content/document.pdf", pages_and_text, 11)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [8]:
len(pages_and_text)

101

In [9]:
df = pd.DataFrame(pages_and_text)
df.describe().round(2)

,page_number,page_char_count,number_of_tokens,pages_sentence_count
count,101.0,101.00,101.00,101.00
mean,43.0,3835.26,958.81,29.84
std,29.3,1910.69,477.67,12.91
min,-7.0,171.00,42.75,5.00
25%,18.0,2284.00,571.00,21.00
50%,43.0,3806.00,951.50,30.00
75%,68.0,5477.00,1369.25,41.00
max,93.0,6729.00,1682.25,52.00


In [11]:
random.sample(pages_and_text, k=5)

[{'page_number': 84,
  'page_char_count': 174,
  'number_of_tokens': 43.5,
  'pages_sentence_count': 5,
  'page_text': '  a brief history of time - stephen hawking... chapter 11 file:///c|/windows/desktop/blahh/stephen hawking - a brief history of time/j.html (11 of 11) [2/20/2001 3:15:56 am] ',
  'Book_Name': '/content/Stephen Hawking -A Brief History Of Time '},
 {'page_number': 63,
  'page_char_count': 2917,
  'number_of_tokens': 729.25,
  'pages_sentence_count': 23,
  'page_text': 'additional red shift. further predictions of the no boundary condition are currently being worked out. a particularly interesting problem is the size of the small departures from uniform density in the early universe that caused the formation first of the galaxies, then of stars, and finally of us. the uncertainty principle implies that the early universe cannot have been completely uniform because there must have been some uncertainties or fluctuations in the positions and velocities of the particles. u

In [12]:
from spacy.lang.en import English

nlp = English()
nlp.add_pipe("sentencizer")

In [13]:
for item in tqdm(pages_and_text):
    page_text = item["page_text"]
    item["pages_sentences"] = page_text.split(".")
    doc = nlp(page_text)
    spacy_sentences = list(doc.sents)
    item["pages_sentences_spacy"] = [str(sen) for sen in spacy_sentences]
    item["spacy_sentence_count"] = len(item["pages_sentences_spacy"])

  0%|          | 0/101 [00:00<?, ?it/s]

In [14]:
pages_and_text[12]

{'page_number': 5,
 'page_char_count': 1233,
 'number_of_tokens': 308.25,
 'pages_sentence_count': 14,
 'page_text': 'figure 2:1 using this procedure, observers who are moving relative to each other will assign different times and positions to the same event. no particular observer’s measurements are any more correct than any other observer’s, but all the measurements are related. any observer can work out precisely what time and position any other observer will assign to an event, provided he knows the other observer’s relative velocity. nowadays we use just this method to measure distances precisely, because we can measure time more accurately than length. in effect, the meter is defined to be the distance traveled by light in 0.000000003335640952 second, as measured by a cesium clock. (the reason for that particular number is that it corresponds to the historical definition of the meter – in terms of two marks on a particular platinum bar kept in paris.) equally, we can use a more c

In [15]:
df = pd.DataFrame(pages_and_text)
df.describe().round(2)

,page_number,page_char_count,number_of_tokens,pages_sentence_count,spacy_sentence_count
count,101.0,101.00,101.00,101.00,101.00
mean,43.0,3835.26,958.81,29.84,27.12
std,29.3,1910.69,477.67,12.91,13.43
min,-7.0,171.00,42.75,5.00,1.00
25%,18.0,2284.00,571.00,21.00,17.00
50%,43.0,3806.00,951.50,30.00,28.00
75%,68.0,5477.00,1369.25,41.00,38.00
max,93.0,6729.00,1682.25,52.00,48.00


In [16]:
CHUNK_SIZE = 10


def split_list(input_list: list, slice_size: int) -> list[list[str]]:

    return [
        input_list[i : i + slice_size] for i in range(0, len(input_list), slice_size)
    ]


for item in tqdm(pages_and_text):
    item["sentence_chunks"] = split_list(
        input_list=item["pages_sentences_spacy"], slice_size=CHUNK_SIZE
    )
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/101 [00:00<?, ?it/s]

In [17]:
pages_and_text[12]

{'page_number': 5,
 'page_char_count': 1233,
 'number_of_tokens': 308.25,
 'pages_sentence_count': 14,
 'page_text': 'figure 2:1 using this procedure, observers who are moving relative to each other will assign different times and positions to the same event. no particular observer’s measurements are any more correct than any other observer’s, but all the measurements are related. any observer can work out precisely what time and position any other observer will assign to an event, provided he knows the other observer’s relative velocity. nowadays we use just this method to measure distances precisely, because we can measure time more accurately than length. in effect, the meter is defined to be the distance traveled by light in 0.000000003335640952 second, as measured by a cesium clock. (the reason for that particular number is that it corresponds to the historical definition of the meter – in terms of two marks on a particular platinum bar kept in paris.) equally, we can use a more c

In [18]:
pages_and_chunks = []
for item in tqdm(pages_and_text):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(
            r"\.([A-Z])", r". \1", joined_sentence_chunk
        )  # ".A" -> ". A" for any full-stop/capital letter combo
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len(
            [word for word in joined_sentence_chunk.split(" ")]
        )
        chunk_dict["chunk_token_count"] = (
            len(joined_sentence_chunk) / 4
        )  # 1 token = ~4 characters

        pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

  0%|          | 0/101 [00:00<?, ?it/s]

317

In [19]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,317.00,317.00,317.00,317.00
mean,43.70,1213.28,202.12,303.32
std,28.43,388.78,67.27,97.20
min,-7.00,168.00,24.00,42.00
25%,20.00,1006.00,167.00,251.50
50%,46.00,1276.00,215.00,319.00
75%,66.00,1472.00,246.00,368.00
max,93.00,2077.00,352.00,519.25


In [21]:
!!pip install --q sentence_transformers

['\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/227.1 kB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b \x1b225.3/227.1 kB\x1b \x1b9.1 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b227.1/227.1 kB\x1b \x1b6.7 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25h\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/21.3 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b4.9/21.3 MB\x1b \x1b148.4 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━\x1b \x1b11.1/21.3 MB\x1b \x1b164.0 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╺\x1b\x1b━━━━━━\x1b \x1b17.6/21.3 MB\x1b \x1b182.6 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b \x1b2

In [22]:
from sentence_transformers import SentenceTransformer, util


MODEL_NAME = "all-mpnet-base-v2"

embedding_model = SentenceTransformer(model_name_or_path=MODEL_NAME, device="cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
for item in tqdm(pages_and_chunks):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"], batch_size=64)

  0%|          | 0/317 [00:00<?, ?it/s]

In [24]:
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks)
embeddings_df_save_path = "chunks_and_embeddings.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [25]:
pages_and_chunks = text_chunks_and_embeddings_df.to_dict(orient="records")
embeddings = text_chunks_and_embeddings_df["embedding"].to_list()
tensor_embeddings = [torch.Tensor(embedding) for embedding in embeddings]
embedding_tensor = torch.stack(tensor_embeddings).to(device)

In [26]:
# let's look embedding tensor
embedding_tensor

tensor([[ 0.0215, -0.0004, -0.0059,  ...,  0.0565, -0.0205, -0.0440],
        [ 0.0193, -0.0143, -0.0091,  ...,  0.0389, -0.0522, -0.0451],
        [-0.0066,  0.0007, -0.0017,  ...,  0.0580, -0.0014, -0.0146],
        ...,
        [-0.0067, -0.0551, -0.0076,  ..., -0.0088, -0.0424, -0.0305],
        [-0.0021, -0.0826, -0.0282,  ..., -0.0142,  0.0315, -0.0265],
        [ 0.0350,  0.0212, -0.0156,  ...,  0.0574, -0.0422, -0.0377]],
       device='cuda:0')

In [27]:
query = "what are black holes"
print(f"Query: {query}")

query_embedding = embedding_model.encode(query, convert_to_tensor=True)

from time import perf_counter as timer

start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=embedding_tensor)[0]
end_time = timer()

print(
    f"Time take to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds."
)

top_results_dot_product = torch.topk(dot_scores, k=5)
print(top_results_dot_product)

Query: what are black holes
Time take to get scores on 317 embeddings: 0.00322 seconds.
torch.return_types.topk(
values=tensor([0.6186, 0.5986, 0.5203, 0.5191, 0.5170], device='cuda:0'),
indices=tensor([127, 156, 141, 148, 176], device='cuda:0'))


In [28]:
print(pages_and_chunks[148]["sentence_chunk"])

but they are all too far away and therefore too difficult to observe to provide conclusive evidence of black holes.further encouragement for the existence of black holes came in 1967 with the discovery by a research student at cambridge, jocelyn bell-burnell, of objects in the sky that were emitting regular pulses of radio waves.at first bell and her supervisor, antony hewish, thought they might have made contact with an alien civilization in the galaxy!indeed, at the seminar at which they announced their discovery, i remember that they called the first four sources to be found lgm 1 – 4, lgm standing for “little green men.”in the end, however, they and everyone else came to the less romantic conclusion that these objects, which were given the name pulsars, were in fact rotating neutron stars that were emitting pulses of radio waves because of a complicated interaction between their magnetic fields and surrounding matter.this was bad news for writers of space westerns, but very hopeful

In [29]:
from time import perf_counter as timer

query = "what happens to a human when he/she enters a black hole"
print(f"Query: {query}")


query_embedding = embedding_model.encode(query, convert_to_tensor=True)

start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=embedding_tensor)[0]
end_time = timer()

print(
    f"Time take to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds."
)

top_results_dot_product = torch.topk(dot_scores, k=5)
print(top_results_dot_product)

Query: what happens to a human when he/she enters a black hole
Time take to get scores on 317 embeddings: 0.00016 seconds.
torch.return_types.topk(
values=tensor([0.6485, 0.5607, 0.5341, 0.5077, 0.4755], device='cuda:0'),
indices=tensor([141, 139, 177, 176, 137], device='cuda:0'))


In [30]:
pages_and_chunks[164]

{'page_number': 47,
 'sentence_chunk': 'they convinced me that, according to the quantum mechanical uncertainty principle, rotating black holes should create and emit particles.i believed their arguments on physical grounds, but i did not like the mathematical way in which they calculated the emission.i therefore set about devising a better mathematical treatment, which i described at an informal seminar in oxford at the end of november 1973.at that time i had not done the calculations to find out how much would actually be emitted.i was expecting to discover just the radiation that zeldovich and starobinsky had predicted from rotating black holes.however, when i did the calculation, i a brief history of time - stephen hawking... chapter 7 file:///c|/windows/desktop/blahh/stephen hawking - a brief history of time/f.html (3 of 8) [2/20/2001 3:15:18 am]',
 'chunk_char_count': 823,
 'chunk_word_count': 126,
 'chunk_token_count': 205.75,
 'embedding': array([ 2.97900923e-02, -3.39133032e-0

In [31]:
def retrival_query_resources(
    query: str,
    embeddings: torch.tensor,
    model: SentenceTransformer = embedding_model,
    indices_to_return: int = 5,
):
    query_embedding = model.encode(query, convert_to_tensor=True)
    start_time = timer()
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    end_time = timer()
    print(
        f" Time taken to get scores on {len(embeddings)} embeddings :{end_time - start_time}"
    )
    scores, indices = torch.topk(dot_scores, k=indices_to_return)
    return scores, indices


def print_query_results(query: str, embeddings, pages_and_chunks=pages_and_chunks):
    scores, indices = retrival_query_resources(query=query, embeddings=embeddings)

    for score, idx in zip(scores, indices):
        print(f"Score:{score*100:.2f}")
        print("Text:")
        print(pages_and_chunks[idx]["sentence_chunk"])
        print("\n")

In [32]:
print_query_results("what is Black Hole", embedding_tensor)

 Time taken to get scores on 317 embeddings :8.695199994690483e-05
Score:57.54
Text:
chapter 7 black holes ain’t so black  before 1970, my research on general relativity had concentrated mainly on the question of whether or not there had been a big bang singularity.however, one evening in november that year, shortly after the birth of my daughter, lucy, i started to think about black holes as i was getting into bed.my disability makes this rather a slow process, so i had plenty of time.at that date there was no precise definition of which points in space-time lay inside a black hole and which lay outside.i had already discussed with roger penrose the idea of defining a black hole as the set of events from which it was not possible to escape to a large distance, which is now the generally accepted definition.it means that the boundary of the black hole, the event horizon, is formed by the light rays that just fail to escape from the black hole, hovering forever just on the edge figure 7